# Sentinel-2 Super-Resolution (4×)

**Upscale Sentinel-2 imagery from 10m/pixel → 2.5m/pixel using SwinIR**

- ✅ **Streaming Inference**: Fetches live data from GEE (No massive downloads)
- ✅ **SwinIR Model**: Pretrained deep swin transformer for restoration
- ✅ **Hallucination Guardrails**: Ensures geospatial consistency

Enable GPU: `Runtime → Change runtime type → T4 GPU` (Recommended for speed)

In [ ]:
# 1. Clone repository & install
!git clone https://github.com/Rishikarnatakam/Klymo.git
%cd Klymo
!pip install -q -r requirements.txt
!pip install streamlit pyngrok localtunnel

In [ ]:
# 2. Check Device
import torch
if torch.cuda.is_available():
    print(f"✓ GPU Detected: {torch.cuda.get_device_name(0)}")
    DEVICE = 'cuda'
else:
    print("⚠ No GPU found. Using CPU (Inference will be slower).")
    DEVICE = 'cpu'

In [ ]:
# 3. GEE Authentication
from google.colab import auth
auth.authenticate_user()

import ee
ee.Authenticate()
ee.Initialize(project='klymo-486313')
print("✓ GEE authenticated")

In [ ]:
# 4. Run Static Inference (Quick Check)
# This runs the pipeline once to verify everything works and generate metrics
from src.models.swinir import load_swinir_model
from src.data.gee_fetcher import GEEFetcher
from src.inference.pipeline import SuperResolutionPipeline
from src.metrics.psnr import compute_psnr
from src.metrics.ssim import compute_ssim
import matplotlib.pyplot as plt
from src.data.preprocessing import to_8bit_visualization
from skimage.transform import resize

# 1. Fetch Tile
fetcher = GEEFetcher(authenticate=False)
fetcher.authenticated = True
fetcher.ee = ee
delhi_tile = fetcher.fetch_tile('delhi', tile_size=256)

# 2. Run Pipeline
pipeline = SuperResolutionPipeline(device=DEVICE)
results = pipeline.run(delhi_tile)

# 3. Metrics
psnr = compute_psnr(results['bicubic'], results['sr'])
ssim = compute_ssim(results['bicubic'], results['sr'])
print(f"PSNR: {psnr:.2f} dB | SSIM: {ssim:.4f}")

# 4. Show Result
fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(to_8bit_visualization(results['bicubic']))
axes[0].set_title('Bicubic (Baseline)')
axes[1].imshow(to_8bit_visualization(results['sr']))
axes[1].set_title(f'SwinIR (SR) | PSNR: {psnr:.2f}')
plt.show()

In [ ]:
# 5. 🚀 Launch Interactive App (Streamlit)
# Use the URL printed below to view the interactive slider demo
# Password is usually the IP address shown in the output

print("Starting Streamlit App...")
!streamlit run app/streamlit_app.py & npx localtunnel --port 8501